# IMPORTS

In [22]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import math, copy
import torch.optim as optim

device = torch.device("cuda")
print(device)

cuda


In [20]:
model_cifar10 = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg16_bn", pretrained=False).to(device)
model_cifar100 = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_vgg16_bn", pretrained=False).to(device)

Using cache found in /home/foxunderground/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master
Using cache found in /home/foxunderground/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


In [21]:
from torch.profiler import profile, ProfilerActivity

model_cifar10.eval()
inputs = torch.randn(256, 3, 32, 32).to(device)

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA] if torch.cuda.is_available() else [ProfilerActivity.CPU]) as prof:
    with torch.no_grad():
        model_cifar10(inputs)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

model_cifar100.eval()
inputs = torch.randn(256, 3, 32, 32).to(device)

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA] if torch.cuda.is_available() else [ProfilerActivity.CPU]) as prof:
    with torch.no_grad():
        model_cifar100(inputs)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                  cudaDeviceSynchronize        85.22%      24.314ms        85.22%      24.314ms      24.314ms       0.000us         0.00%       0.000us       0.000us             1  
                                           aten::conv2d         0.11%      29.960us         8.27%       2.360ms     181.566us       0.000us         0.00%      16.771ms       1.290ms            13  
         

In [23]:
transform_train_c10 = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                        (0.2023, 0.1994, 0.2010))
])

transform_test_c10 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

trainset_c10 = torchvision.datasets.CIFAR10(root='./data/cifar10', train=False, download=True, transform=transform_train_c10)
testset_c10 = torchvision.datasets.CIFAR10(root='./data/cifar10', train=False, download=True, transform=transform_test_c10)
trainloader_c10 = DataLoader(trainset_c10, batch_size=64, shuffle=True, num_workers=0)
testloader_c10 = DataLoader(testset_c10, batch_size=64, shuffle=False, num_workers=0)

In [24]:
mean_c100 = [0.5070, 0.4865, 0.4409]
std_c100 = [0.2673, 0.2564, 0.2761]

train_transform_c100 = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean_c100, std_c100)
])

test_transform_c100 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_c100, std_c100)
])

trainset_c100 = torchvision.datasets.CIFAR100(root="./data/cifar100", train=True, download=True, transform=train_transform_c100)
testset_c100 = torchvision.datasets.CIFAR100(root="./data/cifar100", train=False, download=True, transform=test_transform_c100)

trainloader_c100 = DataLoader(trainset_c100, batch_size=64, shuffle=True, num_workers=0)
testloader_c100 = DataLoader(testset_c100, batch_size=64, shuffle=False, num_workers=0)

# Training

In [6]:
lr = 1e-3
optimizer = torch.optim.SGD(model_cifar10.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

model_cifar10.train()
acc = 0
epoch = 0
while acc < 20:
    epoch += 1
    running_loss, correct, total = 0, 0, 0
    for x, y in tqdm(trainloader_c10, desc=f"Epoch {epoch}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model_cifar10(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, pred = out.max(1)
        total += y.size(0)
        correct += pred.eq(y).sum().item()
        acc = 100*(correct/total)
    print(f"Epoch {epoch}: Loss={running_loss/len(trainloader_c10):.4f}, Acc={(100*correct/total):.2f}%")

Epoch 1:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 1: Loss=2.3021, Acc=10.43%


Epoch 2:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 2: Loss=2.3004, Acc=10.51%


Epoch 3:   0%|          | 0/157 [00:00<?, ?it/s]

...

Epoch 6: Loss=2.0809, Acc=18.51%


Epoch 7:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 7: Loss=1.9588, Acc=18.66%


Epoch 8:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 8: Loss=1.8962, Acc=20.36%


In [7]:
lr = 1e-3
optimizer = torch.optim.SGD(model_cifar100.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

model_cifar100.train()
acc = 0
epoch = 0
while acc < 20:
    epoch += 1
    running_loss, correct, total = 0, 0, 0
    for x, y in tqdm(trainloader_c100, desc=f"Epoch {epoch}"):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model_cifar100(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, pred = out.max(1)
        total += y.size(0)
        correct += pred.eq(y).sum().item()
        acc = 100*(correct/total)
    print(f"Epoch {epoch}: Loss={running_loss/len(trainloader_c10):.4f}, Acc={(100*correct/total):.2f}%")

Epoch 1:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 1: Loss=22.9322, Acc=1.16%


Epoch 2:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 2: Loss=22.6104, Acc=1.23%


Epoch 3:   0%|          | 0/782 [00:00<?, ?it/s]

...

Epoch 15: Loss=15.8945, Acc=17.88%


Epoch 16:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 16: Loss=15.3737, Acc=19.76%


Epoch 17:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 17: Loss=14.9094, Acc=21.86%


In [8]:
torch.save(model_cifar10.state_dict(), "vgg16bn_cifar10_twenty.pth")
torch.save(model_cifar100.state_dict(), "vgg16bn_cifar100_twenty.pth")

# Task Requirements - CIFAR10

In [9]:
calib_samples = 512
batch_for_calib = 32
target_sparsity = 0.8
stages = [0.5, 0.75, 1.0]
stage_acc_milestones = [40.0, 60.0, None]   # percent
epochs_after_stage = 5
lr = 1e-3
momentum = 0.9
weight_decay = 5e-4

model = model_cifar10.to(device)
model.load_state_dict(torch.load("vgg16bn_cifar10_twenty.pth", map_location=device))

<All keys matched successfully>

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def evaluate(model, loader, device):
    model.eval()
    top1 = 0
    top5 = 0
    total = 0
    with torch.no_grad():
        for x,y in loader:
            x,y = x.to(device), y.to(device)
            out = model(x)
            _, pred = out.topk(5, dim=1)
            y_expand = y.unsqueeze(1).expand_as(pred)
            top1 += (pred[:,0] == y).sum().item()
            top5 += (pred == y_expand).any(dim=1).sum().item()
            total += y.size(0)
    return 100.0*top1/total, 100.0*top5/total

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for x,y in tqdm(loader, leave=False):
        x,y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * x.size(0)
        pred = out.argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)
    return running_loss/total, 100.0*correct/total

def grasp_prune_global(model, dataloader, device, keep_ratio=0.5, num_batches=5):
    model_cp = copy.deepcopy(model).to(device)
    model_cp.train()
    criterion = nn.CrossEntropyLoss()
    weight_params = [p for p in model_cp.parameters() if p.requires_grad and p.dim() > 1]
    grads_sum = None
    used = 0

    for i, (x, y) in enumerate(dataloader):
        if i >= num_batches: break
        x, y = x.to(device), y.to(device)
        out = model_cp(x)
        loss = criterion(out, y)
        grads = torch.autograd.grad(loss, weight_params, create_graph=True, retain_graph=True, allow_unused=True)
        if grads_sum is None:
            grads_sum = [g.detach().clone() if g is not None else torch.zeros_like(p) for g,p in zip(grads, weight_params)]
        else:
            for j,g in enumerate(grads):
                if g is None:
                    continue
                grads_sum[j] += g.detach()
        used += 1

    grads_mean = [g / float(used) for g in grads_sum]
    model_cp.zero_grad()
    tot_loss = 0.0
    tot_n = 0

    for i, (x,y) in enumerate(dataloader):
        if i >= num_batches: break
        x,y = x.to(device), y.to(device)
        out = model_cp(x)
        tot_loss = tot_loss + nn.CrossEntropyLoss(reduction='sum')(out, y)
        tot_n += x.size(0)
    tot_loss = tot_loss / float(tot_n)
    grads_with_graph = torch.autograd.grad(tot_loss, weight_params, create_graph=True, retain_graph=True, allow_unused=True)
    dot = 0.0

    for gw, gm in zip(grads_with_graph, grads_mean):
        if gw is None:
            continue
        dot = dot + (gw * gm.to(gw.device)).sum()
    hvp = torch.autograd.grad(dot, weight_params, retain_graph=False, allow_unused=True)
    hvp = [h if h is not None else torch.zeros_like(p) for h,p in zip(hvp, weight_params)]
    all_scores = []
    shapes = []

    for g,h,p in zip(grads_mean, hvp, weight_params):
        g_cpu = g.detach().cpu().reshape(-1)
        h_cpu = h.detach().cpu().reshape(-1)
        s = (-g_cpu * h_cpu)
        all_scores.append(s)
        shapes.append((p.shape, id(p)))
    all_scores_flat = torch.cat(all_scores)
    k = max(1, int(all_scores_flat.numel() * keep_ratio))
    topk_vals, topk_idx = torch.topk(all_scores_flat, k, largest=True)
    thr = topk_vals.min()
    mask_flat = (all_scores_flat >= thr).float()
    mask_dict = {}
    idx = 0
    weight_params_orig = [p for p in model.parameters() if p.requires_grad and p.dim() > 1]

    for (shape, pid), p_orig in zip(shapes, weight_params_orig):
        numel = int(torch.tensor(shape).prod().item())
        layer_mask_flat = mask_flat[idx: idx+numel]
        idx += numel
        layer_mask = layer_mask_flat.reshape(shape).to(p_orig.device)
        mask_dict[pid] = layer_mask.cpu()
    idx = 0

    for p in [p for p in model_cp.parameters() if p.requires_grad and p.dim() > 1]:
        pid = id(p)
        if pid in mask_dict:
            m = mask_dict[pid].to(p.device)
            with torch.no_grad():
                p.mul_(m)
            p.weight_mask = m
        idx += 1
    return model_cp, mask_dict

In [11]:
TARGET_SPARSITY = 0.7
CALIBRATION_BATCHES = 10
MAX_RETRAIN_EPOCHS_PER_STAGE = 30
base_name = "cifar10_vgg16bn"
trainloader = trainloader_c10
testloader = testloader_c10

print("Dataset:", base_name)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

val1, val5 = evaluate(model, testloader, device)
print("Baseline val top1/top5:", val1, val5)


Dataset: cifar10_vgg16bn
Baseline val top1/top5: 23.17 85.37


In [12]:

cumulative_masks = {}
stages = [0.5, 0.75, 1.0]

for stage_idx, frac in enumerate(stages, 1):
    desired_sparsity = frac * TARGET_SPARSITY
    keep_ratio = 1.0 - desired_sparsity
    print(f"\nStage {stage_idx}: desired global sparsity {desired_sparsity:.3f} -> keep_ratio {keep_ratio:.3f}")
    pruned_model, new_masks = grasp_prune_global(model, trainloader, device, keep_ratio=keep_ratio, num_batches=CALIBRATION_BATCHES)

    for pid, m_new in new_masks.items():
        if pid in cumulative_masks:
            cumulative_masks[pid] = (cumulative_masks[pid] * m_new).cpu()
        else:
            cumulative_masks[pid] = m_new.cpu()

    with torch.no_grad():
        for p in [p for p in model.parameters() if p.requires_grad and p.dim() > 1]:
            pid = id(p)
            if pid in cumulative_masks:
                p.mul_(cumulative_masks[pid].to(p.device))
                p.weight_mask = cumulative_masks[pid].to(p.device)

    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
    target_acc = [40.0, 60.0, None][stage_idx-1]
    if target_acc is not None:
        epoch = 0
        while True:
            epoch += 1
            loss, acc = train_one_epoch(model, trainloader, optimizer, criterion, device)
            val1, val5 = evaluate(model, testloader, device)
            print(f" Retrain ep{epoch}: train acc={acc:.2f} val acc={val1:.2f}")
            with torch.no_grad():
                for p in model.parameters():
                    if hasattr(p, "weight_mask"):
                        p.mul_(p.weight_mask.to(p.device))
            if val1 >= target_acc or epoch >= MAX_RETRAIN_EPOCHS_PER_STAGE:
                break
    else:
        optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
        for e in range(5):
            train_one_epoch(model, trainloader, optimizer, criterion, device)
            val1, _ = evaluate(model, testloader, device)
            print(f"  post-final train epoch {e+1}: val1 {val1:.2f}")
    torch.save(model.state_dict(), f"{base_name}_grasp_stage{stage_idx}.pth")


Stage 1: desired global sparsity 0.350 -> keep_ratio 0.650


  0%|          | 0/157 [00:00<?, ?it/s]

 Retrain ep1: train acc=24.12 val acc=29.33


  0%|          | 0/157 [00:00<?, ?it/s]

 Retrain ep2: train acc=32.63 val acc=37.96


...

  post-final train epoch 3: val1 69.01


  0%|          | 0/157 [00:00<?, ?it/s]

  post-final train epoch 4: val1 69.54


  0%|          | 0/157 [00:00<?, ?it/s]

  post-final train epoch 5: val1 71.67


In [13]:
val1, val5 = evaluate(model, testloader, device)
print("Final val top1/top5:", val1, val5)
torch.save(model.state_dict(), f"{base_name}_grasp_final.pth")


Final val top1/top5: 71.67 97.88


# Task Requirements - CIFAR100

In [14]:
calib_samples = 512
batch_for_calib = 32
target_sparsity = 0.8
stages = [0.5, 0.75, 1.0]
stage_acc_milestones = [40.0, 60.0, None]
epochs_after_stage = 5
lr = 1e-3
momentum = 0.9
weight_decay = 5e-4

model = model_cifar100.to(device)
model.load_state_dict(torch.load("vgg16bn_cifar100_twenty.pth", map_location=device))

<All keys matched successfully>

In [15]:
TARGET_SPARSITY = 0.7
CALIBRATION_BATCHES = 10
MAX_RETRAIN_EPOCHS_PER_STAGE = 30
base_name = "cifar100_vgg16bn"
trainloader = trainloader_c100
testloader = testloader_c100

print("Dataset:", base_name)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

val1, val5 = evaluate(model, testloader, device)
print("Baseline val top1/top5:", val1, val5)


Dataset: cifar100_vgg16bn
Baseline val top1/top5: 23.45 55.65


In [16]:

cumulative_masks = {}
stages = [0.5, 0.75, 1.0]

for stage_idx, frac in enumerate(stages, 1):
    desired_sparsity = frac * TARGET_SPARSITY
    keep_ratio = 1.0 - desired_sparsity
    print(f"\nStage {stage_idx}: desired global sparsity {desired_sparsity:.3f} -> keep_ratio {keep_ratio:.3f}")
    pruned_model, new_masks = grasp_prune_global(model, trainloader, device, keep_ratio=keep_ratio, num_batches=CALIBRATION_BATCHES)

    for pid, m_new in new_masks.items():
        if pid in cumulative_masks:
            cumulative_masks[pid] = (cumulative_masks[pid] * m_new).cpu()
        else:
            cumulative_masks[pid] = m_new.cpu()

    with torch.no_grad():
        for p in [p for p in model.parameters() if p.requires_grad and p.dim() > 1]:
            pid = id(p)
            if pid in cumulative_masks:
                p.mul_(cumulative_masks[pid].to(p.device))
                p.weight_mask = cumulative_masks[pid].to(p.device)

    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
    target_acc = [40.0, 60.0, None][stage_idx-1]
    if target_acc is not None:
        epoch = 0
        while True:
            epoch += 1
            loss, acc = train_one_epoch(model, trainloader, optimizer, criterion, device)
            val1, val5 = evaluate(model, testloader, device)
            print(f" Retrain ep{epoch}: train acc={acc:.2f} val acc={val1:.2f}")
            with torch.no_grad():
                for p in model.parameters():
                    if hasattr(p, "weight_mask"):
                        p.mul_(p.weight_mask.to(p.device))
            if val1 >= target_acc or epoch >= MAX_RETRAIN_EPOCHS_PER_STAGE:
                break
    else:
        optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
        for e in range(5):
            train_one_epoch(model, trainloader, optimizer, criterion, device)
            val1, _ = evaluate(model, testloader, device)
            print(f"  post-final train epoch {e+1}: val1 {val1:.2f}")
    torch.save(model.state_dict(), f"{base_name}_grasp_stage{stage_idx}.pth")



Stage 1: desired global sparsity 0.350 -> keep_ratio 0.650


  0%|          | 0/782 [00:00<?, ?it/s]

 Retrain ep1: train acc=21.73 val acc=25.87


  0%|          | 0/782 [00:00<?, ?it/s]

 Retrain ep2: train acc=25.61 val acc=27.85


...

  post-final train epoch 3: val1 61.04


  0%|          | 0/782 [00:00<?, ?it/s]

  post-final train epoch 4: val1 59.64


  0%|          | 0/782 [00:00<?, ?it/s]

  post-final train epoch 5: val1 60.54


In [17]:
val1, val5 = evaluate(model, testloader, device)
print("Final val top1/top5:", val1, val5)
torch.save(model.state_dict(), f"{base_name}_grasp_final.pth")


Final val top1/top5: 60.54 85.02


# Final Results

In [25]:
import torch
import time
import io
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetTotalEnergyConsumption
import thop.profile
from torch.profiler import profile, record_function, ProfilerActivity

In [26]:
def evaluate_model_efficiency(model, dataloader, device, runs=100):
    model.to(device)
    model.eval()

    data_iter = iter(dataloader)
    images, labels = next(data_iter)
    images, labels = images.to(device), labels.to(device)

    nvmlInit()
    gpu = nvmlDeviceGetHandleByIndex(0)
    start_energy = nvmlDeviceGetTotalEnergyConsumption(gpu)

    total_time, total_mem, correct1, correct5 = 0, 0, 0, 0
    for _ in range(runs):
        torch.cuda.reset_peak_memory_stats()
        start_t = time.time()
        with torch.no_grad():
            outputs = model(images)
        torch.cuda.synchronize()
        end_t = time.time()
        total_time += (end_t - start_t) * 1000
        total_mem += torch.cuda.memory_allocated(device) / (1024 ** 2)
        top5_pred = outputs.topk(5, dim=1).indices
        top1_pred = outputs.argmax(dim=1)
        correct1 += (top1_pred == labels).sum().item()
        correct5 += sum(labels[i] in top5_pred[i] for i in range(len(labels)))
    total = len(labels)
    top1_acc = 100 * correct1 / (total * runs)
    top5_acc = 100 * correct5 / (total * runs)

    end_energy = nvmlDeviceGetTotalEnergyConsumption(gpu)
    avg_energy = (end_energy - start_energy) / runs
    avg_latency = total_time / runs
    avg_mem = total_mem / runs
    peak_mem = torch.cuda.max_memory_allocated(device) / (1024 ** 2)

    buf = io.BytesIO()
    torch.save(model.state_dict(), buf)
    model_size = len(buf.getvalue()) / (1024 ** 2)

    try:
        macs, _ = thop.profile(model, inputs=(images,), verbose=False)
    except Exception:
        macs = float("nan")
    print("\n========== Profiling Summary ==========")
    print(f"Runs:            {runs}")
    print(f"Model Size:      {model_size:.2f} MB")
    print(f"Average Latency: {avg_latency:.2f} ms")
    print(f"Peak Memory:     {peak_mem:.2f} MB")
    print(f"Mean Memory:     {avg_mem:.2f} MB")
    print(f"Energy / Run:    {avg_energy:.2f} mJ")
    print(f"Top-1 Accuracy:  {top1_acc:.2f}%")
    print(f"Top-5 Accuracy:  {top5_acc:.2f}%")
    print(f"MACs / Batch:    {macs / 1e6:.2f} M")
    print("=======================================\n")

    return {
        "model_size_mb": model_size,
        "latency_ms": avg_latency,
        "avg_mem_mb": avg_mem,
        "peak_mem_mb": peak_mem,
        "energy_mJ": avg_energy,
        "top1_acc": top1_acc,
        "top5_acc": top5_acc,
        "macs": macs
    }


In [27]:
model_cifar10.load_state_dict(torch.load("cifar10_vgg16bn_grasp_final.pth"))
model_cifar100.load_state_dict(torch.load("cifar100_vgg16bn_grasp_final.pth"))

<All keys matched successfully>

In [28]:
device = torch.device("cuda")
evaluation_c10 = evaluate_model_efficiency(model_cifar10, testloader_c10, device)

model_cifar10.eval()
inputs = torch.randn(256, 3, 32, 32).to(device)
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA] if torch.cuda.is_available() else [ProfilerActivity.CPU]) as prof:
    with torch.no_grad():
        model_cifar10(inputs)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))


time.sleep(3)
evaluation_c100 = evaluate_model_efficiency(model_cifar100, testloader_c100, device)

model_cifar100.eval()
inputs = torch.randn(256, 3, 32, 32).to(device)
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA] if torch.cuda.is_available() else [ProfilerActivity.CPU]) as prof:
    with torch.no_grad():
        model_cifar100(inputs)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))


========== Profiling Summary ==========
Runs:            100
Model Size:      58.25 MB
Average Latency: 8.53 ms
Peak Memory:     354.16 MB
Mean Memory:     249.60 MB
Energy / Run:    2458.44 mJ
Top-1 Accuracy:  82.81%
Top-5 Accuracy:  98.44%
MACs / Batch:    20149.24 M

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                  cudaDeviceSynchronize        87.65%      16.755ms        87.65%      16.755ms      16.755ms       0